In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from intersectomics.utils import add_cols_multi_index
from intersectomics.bootstrap_corr import bootstrap_spearman_corr_parrallel
from intersectomics.plot import plot_time_single_omics_layer, plot_time_multiple_omics_layers
from intersectomics.graph import make_intersection_graph

In this example, we will use the data extracted from the following [paper](https://www.sciencedirect.com/science/article/pii/S0048969723003558) and calculate all spearman correlations between all possible pairs of transcripts and all possible pairs of proteins considering that the data contains replicates. From that, we will build a graph where the nodes are the transcripts/proteins and the nodes are the correlations values. We use a p-value of 0.05 as a cutoff to remove any insignificant results.

The two graphs are then intersected to a combined graph that represents the significant correlation between the two data types. Finally, we perform a community analysis on the intersected graph and extract lists of genes/proteins that behave similarly. 

Note: One limitation of this method is the need for the index to match. That means you need a 1:1 match between a transcript name and a protein name.

In [ ]:
# Read the data
# in this example we use a subset of the full dataset to make things go faster
norm_rna_seq = pd.read_csv('data/rna_norm_counts_small.csv.xz', index_col=0)
metadata_rna_seq = pd.read_csv('data/rna_metadata.csv')

# Combine the metadata and the coutn table for RNA-seq
df_rna_seq = add_cols_multi_index(norm_rna_seq, metadata_rna_seq, 'series')
# For this example, use the control and not treated samples
df_rna_seq = df_rna_seq.loc[:,df_rna_seq.columns.get_level_values('treatment')=='control']

In [ ]:
# Filter lowly expressed genes using CPM (counts per million)
# TODO: use preset method in R
total_counts = df_rna_seq.sum()
cpm = df_rna_seq.divide(total_counts) * 1e6
filtered_data = cpm[cpm > 1].dropna()
cpm = None
filtered_data = filtered_data.fillna(0.0)
df_rna_seq = filtered_data
filtered_data = None

In [ ]:
# Read the data
# in this example we use a subset of the full dataset to make things go faster
norm_protein = pd.read_csv('data/protein_norm_counts_small.csv.xz', index_col=0)
metadata_protein = pd.read_csv('data/protein_metadata.csv')

# Combine the metadata and the coutn table for preteins
df_protein = add_cols_multi_index(norm_protein, metadata_protein, 'sample')
# For this example, use the control and not treated samples
df_protein = df_protein.loc[:,df_protein.columns.get_level_values('treatment')=='control']

In [ ]:
# Change the names of the index for proteins to match the RNA-seq
convert_names = pd.read_csv('data/name_convert.csv', index_col=0)
c = convert_names.set_index('protein_name').to_dict()['gene_name']
df_protein.index = [c[i] for i in df_protein.index]

In [ ]:
# Ultimately we would like to study the intersection between the two data types, we 
# can remove any element that is not present in either
df_rna_seq = df_rna_seq.loc[list(set(df_protein.index.unique()) & set(df_rna_seq.index.unique()))]
df_protein = df_protein.loc[list(set(df_protein.index.unique()) & set(df_rna_seq.index.unique()))]

In [ ]:
# Calculate the spearman correlation for all possible pairs of proteins
protein_corr, protein_pvalues = bootstrap_spearman_correlation_parallel(
    df_input=df_protein, 
    replicate_column_name='time', 
    n_processes=cpu_count()-1,
    n_iterations=10,
    chunk_size=10,
)

In [ ]:
# Calculate the spearman correlation for all possible pairs of RNA-seq 
rna_seq_corr, rna_seq_pvalues = bootstrap_spearman_correlation_parallel(
    df_input=df_rna_seq, 
    replicate_column_name='timepoint', 
    n_processes=cpu_count()-1,
    n_iterations=10,
    chunk_size=10,
)

In [ ]:
G_inter, community_dict = make_intersection_graph(
    corrs=[rna_seq_corr, protein_corr],
    pvalues=[rna_seq_pvalues, protein_pvalues],
    
)

In [ ]:
import numpy as np
import matplotlib.cm as cm

comm = np.unique([community_dict[i] for i in community_dict])
cmap = cm.get_cmap('hsv', len(comm))

pos = nx.spring_layout(G_inter)
plt.figure(3,figsize=(45,35)) 
nx.draw(
    G_inter, 
    pos = pos, 
    with_labels = True, 
    node_color = [cmap(community_dict[i]) for i in G_inter.nodes],
)
plt.show()

In [ ]:
df_rna_seq.columns.names = ['Sample', 'series', 'time', 'treatment', 'file', 'condition']
plot_time_multiple_omics_layers(
    dfs=[df_protein, df_rna_seq],
    dfs_names=['Protein', 'RNA-seq'],
    col_replicate_name='time',
    biomolecules=[i for i in community_dict if community_dict[i]==1],
    num_col=2,
    normalize=True,
    plot_height=800,
    plot_width=900,
)